In [1]:
import psycopg2

In [2]:
# docker run --name postgres-db -e POSTGRES_PASSWORD=docker -p 5432:5432 -d postgres

In [3]:
try: 
    conn = psycopg2.connect("host=localhost user=postgres port=5432  password=docker dbname=sparkifydb")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [4]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

In [5]:
conn.set_session(autocommit=True)

In [6]:
cur.execute("""

select count(distinct start_time)
from songplays

""")

In [7]:
cur.fetchone()

(6793,)

In [2]:
import pandas as pd

In [ ]:
from pathlib import Path
import sys,os

In [28]:
log_dfs = []
path = Path(r'./data/log_data').absolute()

for r,d,f in os.walk(path):
    for file in f:
        path = os.path.join(r, file)
        df = pd.read_json(path, lines=True)
        log_dfs.append(df)
log_df = pd.concat(log_dfs, axis=0).reset_index(drop=True)

In [31]:
song_dfs = []
path = Path(r'./data/song_data').absolute()

for r,d,f in os.walk(path):
    for file in f:
        path = os.path.join(r, file)
        df = pd.read_json(path, lines=True)
        song_dfs.append(df)
song_df = pd.concat(song_dfs, axis=0).reset_index(drop=True)

In [45]:
cur.execute("select * from songplays")
row = cur.fetchone()

In [47]:
song_df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AREBBGV1187FB523D2,NaN,NaN,"Houston, TX",Mike Jones (Featuring CJ_ Mello & Lil' Bran),SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),173.66159,0
1,1,AR1Y2PT1187FB5B9CE,27.94017,-82.32547,Brandon,John Wesley,SOLLHMX12AB01846DC,The Emperor Falls,484.62322,0
2,1,AR5KOSW1187FB35FF4,49.80388,15.47491,Dubai UAE,Elena,SOZCTXZ12AB0182364,Setanta matins,269.58322,0
3,1,AR47JEX1187B995D81,37.83721,-94.35868,"Nevada, MO",SUE THOMPSON,SOBLGCN12AB0183212,James (Hold The Ladder Steady),124.86485,1985
4,1,ARDNS031187B9924F0,32.67828,-83.22295,Georgia,Tim Wilson,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco ...,186.48771,2005
...,...,...,...,...,...,...,...,...,...,...
66,1,ARBGXIG122988F409D,37.77916,-122.42005,California - SF,Steel Rain,SOOJPRH12A8C141995,Loaded Like A Gun,173.19138,0
67,1,AREDL271187FB40F44,NaN,NaN,,Soul Mekanik,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,45.66159,0
68,1,ARNNKDK1187B98BBD5,45.80726,15.96760,Zagreb Croatia,Jinx,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),407.37914,0
69,1,AR62SOJ1187FB47BB5,NaN,NaN,,Chase & Status,SOGVQGJ12AB017F169,Ten Tonne,337.68444,2005


In [49]:
t = pd.to_datetime(log_df['ts'], unit='ms')

In [56]:
tuple(time_data.values[0])

(Timestamp('2018-11-18 02:33:21.796000'),
 2,
 18,
 46,
 11,
 2018,
 <function Timestamp.weekday>)

In [52]:
t.apply(lambda d: d.hour)

0        2
1        3
2        3
3        9
4        9
        ..
8051    21
8052    21
8053    21
8054    23
8055    23
Name: ts, Length: 8056, dtype: int64

In [59]:
time_data['weekday']

0       6
1       6
2       6
3       6
4       6
       ..
8051    3
8052    3
8053    3
8054    3
8055    3
Name: weekday, Length: 8056, dtype: int64

In [58]:
# insert time data records
time_data = pd.DataFrame()
time_data['start_time'] = t
time_data['hour'] = t.apply(lambda d: d.hour)
time_data['day'] = t.apply(lambda d: d.day)
time_data['week'] = t.apply(lambda d: d.week)
time_data['month'] = t.apply(lambda d: d.month)
time_data['year'] = t.apply(lambda d: d.year)
time_data['weekday'] = t.apply(lambda d: d.weekday())

In [60]:
t

0      2018-11-18 02:33:21.796
1      2018-11-18 03:21:20.796
2      2018-11-18 03:22:04.796
3      2018-11-18 09:12:13.796
4      2018-11-18 09:39:04.796
                 ...          
8051   2018-11-08 21:42:38.796
8052   2018-11-08 21:45:21.796
8053   2018-11-08 21:48:28.796
8054   2018-11-08 23:33:16.796
8055   2018-11-08 23:59:36.796
Name: ts, Length: 8056, dtype: datetime64[ns]

In [61]:
t.iloc[0]

Timestamp('2018-11-18 02:33:21.796000')

In [32]:
log_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Rokia TraorÃÂ©,Logged In,Stefany,F,0,White,274.88608,free,"Lubbock, TX",PUT,NextSong,1.540708e+12,693,Zen,200,1542508401796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
1,None,Logged In,Tucker,M,0,Garrison,NaN,free,"Oxnard-Thousand Oaks-Ventura, CA",GET,Home,1.540833e+12,555,None,200,1542511280796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",40
2,Camila,Logged In,Tucker,M,1,Garrison,230.81751,free,"Oxnard-Thousand Oaks-Ventura, CA",PUT,NextSong,1.540833e+12,555,Abrazame (Version Acustica),200,1542511324796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",40
3,None,Logged In,Ann,F,0,Banks,NaN,free,"Salt Lake City, UT",GET,Home,1.540896e+12,573,None,200,1542532333796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
4,Carl Thomas,Logged In,Tucker,M,0,Garrison,196.67546,free,"Oxnard-Thousand Oaks-Ventura, CA",PUT,NextSong,1.540833e+12,698,You Ain't Right (Album Version),200,1542533944796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,None,Logged In,Dominick,M,1,Norris,NaN,free,"Los Angeles-Long Beach-Anaheim, CA",GET,Settings,1.540976e+12,365,None,200,1541713358796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",45
8052,Irish Tenors,Logged In,Rylan,M,2,George,187.48036,free,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,393,Danny Boy,200,1541713521796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8053,Justin Bieber,Logged In,Rylan,M,3,George,212.53179,free,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,393,Runaway Love,200,1541713708796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8054,None,Logged In,Dominick,M,0,Norris,NaN,free,"Los Angeles-Long Beach-Anaheim, CA",GET,Home,1.540976e+12,401,None,200,1541719996796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",45
